In [1]:
# !nvidia-smi

In [2]:
# which nvcc
# nvcc --version
# echo $LD_LIBRARY_PATH

In [3]:
# !pip install boto3 xformers huggingface_hub transformers decord
# !pip install --upgrade torch torchvision

In [4]:
import os
# os.getcwd()
os.chdir("/localdisk4/panwla/Projects/park_vlm/Kangaroo")
os.getcwd()

'/localdisk4/panwla/Projects/park_vlm/Kangaroo'

In [5]:
import json
import os
import boto3
import yaml
# from google.colab import drive

# # Mount Google Drive to access API keys
# drive.mount('/content/drive', force_remount=True)

# Load API keys from file
file_path = './.API_KEYS/API_KEYS.yml'

with open(file_path, 'r') as file:
    api_keys = yaml.safe_load(file)

### WANDB Keys
wandb_key = api_keys['WANDB']['Key']
hf_read_api_key = api_keys['HUGGINGFACE']['HF_READ_API_KEY']

# Extract AWS credentials
aws_access_key_id = api_keys['AWS']['AWS_ACCESS_KEY_ID']
aws_secret_access_key = api_keys['AWS']['AWS_SECRET_ACCESS_KEY']

from huggingface_hub import login
login(hf_read_api_key)

# Initialize the S3 client with credentials
s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

/localdisk4/panwla/conda_envs/env3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# import boto3
# import os

# bucket_name = "parkdata"
# local_directory = "./data"  # Local directory for files

# # Initialize S3 resource
# s3 = boto3.resource(
#     "s3",
#     aws_access_key_id=aws_access_key_id,
#     aws_secret_access_key=aws_secret_access_key
# )

# bucket = s3.Bucket(bucket_name)

# # Ensure local directory exists
# os.makedirs(local_directory, exist_ok=True)

# # Download all files from the bucket
# for obj in bucket.objects.all():
#     local_file_path = os.path.join(local_directory, obj.key)

#     # Ensure local folder structure is maintained
#     os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

#     bucket.download_file(obj.key, local_file_path)
#     print(f"Downloaded: {obj.key} -> {local_file_path}")


In [8]:
# import os
# import boto3

# # --- [Optional] If you need explicit credentials, uncomment & fill these in ---
# # aws_access_key_id = "YOUR_ACCESS_KEY_ID"
# # aws_secret_access_key = "YOUR_SECRET_ACCESS_KEY"

# bucket_name = "parkdata"
# prefix = "pickleFiles/"       # The S3 "folder" you want to download
# local_directory = "./data"    # Local directory where files will be saved

# # Initialize S3 resource (optionally include credentials if needed)
# s3 = boto3.resource(
#     "s3"
#     , aws_access_key_id=aws_access_key_id
#     , aws_secret_access_key=aws_secret_access_key
# )

# bucket = s3.Bucket(bucket_name)

# # Ensure the base local directory exists
# os.makedirs(local_directory, exist_ok=True)

# print(f"Downloading files from s3://{bucket_name}/{prefix} to {local_directory}...\n")

# # Filter objects by prefix
# for obj in bucket.objects.filter(Prefix=prefix):
#     # Skip folder placeholder keys (those ending in "/")
#     if obj.key.endswith("/"):
#         print(f"Skipping folder placeholder: {obj.key}")
#         continue

#     # Construct local path
#     local_file_path = os.path.join(local_directory, obj.key)

#     # Ensure local subdirectories exist
#     os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

#     # Download the file
#     bucket.download_file(obj.key, local_file_path)
#     print(f"Downloaded: {obj.key} -> {local_file_path}")

# print("\nDownload complete!")




In [9]:
# import wandb
# wandb.login(key=wandb_key)

In [10]:

from huggingface_hub import login
login(hf_read_api_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [12]:
import os

# Set environment variables based on your bashrc settings
os.environ["JUPYTER_DATA_DIR"] = "/localdisk4/panwla/jupyter_data"
os.environ["HF_HOME"] = "/localdisk4/panwla/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "/localdisk4/panwla/huggingface_cache/transformers"
os.environ["HF_DATASETS_CACHE"] = "/localdisk4/panwla/huggingface_cache/datasets"
os.environ["TORCH_HOME"] = "/localdisk4/panwla/torch"
os.environ["XDG_CACHE_HOME"] = "/localdisk4/panwla_cache/.cache"
os.environ["TMPDIR"] = "/localdisk4/panwla/tmp"
os.environ["TEMP"] = "/localdisk4/panwla/tmp"
os.environ["TMP"] = "/localdisk4/panwla/tmp"

# Create directories if they don't exist
directories = [
    os.environ["JUPYTER_DATA_DIR"],
    os.environ["HF_HOME"],
    os.environ["TRANSFORMERS_CACHE"],
    os.environ["HF_DATASETS_CACHE"],
    os.environ["TORCH_HOME"],
    os.environ["XDG_CACHE_HOME"],
    os.environ["TMPDIR"],
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)


In [13]:
# !export CUDA_VISIBLE_DEVICES=0


In [16]:
# Download the model checkpoint to a local directory
import os
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Ensure HF_HOME & TMPDIR are set (if needed)
os.environ["HF_HOME"] = "/localdisk4/panwla/huggingface_cache"
os.environ["TMPDIR"] = "/localdisk4/panwla/tmp"

checkpoint_path = snapshot_download(
    repo_id="KangarooGroup/kangaroo",
    cache_dir="/localdisk4/panwla/huggingface_cache",
    resume_download=True,
    max_workers=1,  # reduce concurrency to avoid timeouts
)

print("Model checkpoint downloaded to:", checkpoint_path)

tokenizer = AutoTokenizer.from_pretrained("KangarooGroup/kangaroo")

print("Model will be imported now")

# Option 1: Use a specific device (this worked before)
device = torch.device("cuda:0")
model = AutoModelForCausalLM.from_pretrained(
    "KangarooGroup/kangaroo",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
model = model.to(device)

print("Model shifted to cuda successfully")

# Print device mapping information
for name, param in model.named_parameters():
    print(f"{name} is on {param.device}")

# Define terminators
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

# Define video input path and the zero-shot prompt query
video_path = "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4"
query = '''
Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: "Yes" or "No".

Answer:
'''

# Run the model's chat function
out, history = model.chat(
    video_path=video_path,
    query=query,
    tokenizer=tokenizer,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print("Assistant:\n", out)

Fetching 13 files: 100%|██████████| 13/13 [00:00<00:00, 89828.59it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model checkpoint downloaded to: /localdisk4/panwla/huggingface_cache/models--KangarooGroup--kangaroo/snapshots/ded0c00d24ea319ca1cd549bb890dd577f3fed7b
Model will be imported now


A new version of the following files was downloaded from https://huggingface.co/KangarooGroup/kangaroo:
- modeling_kangaroo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



Model shifted to cuda successfully
model.embed_tokens.weight is on cuda:0
model.layers.0.self_attn.q_proj.weight is on cuda:0
model.layers.0.self_attn.k_proj.weight is on cuda:0
model.layers.0.self_attn.v_proj.weight is on cuda:0
model.layers.0.self_attn.o_proj.weight is on cuda:0
model.layers.0.mlp.gate_proj.weight is on cuda:0
model.layers.0.mlp.up_proj.weight is on cuda:0
model.layers.0.mlp.down_proj.weight is on cuda:0
model.layers.0.input_layernorm.weight is on cuda:0
model.layers.0.post_attention_layernorm.weight is on cuda:0
model.layers.1.self_attn.q_proj.weight is on cuda:0
model.layers.1.self_attn.k_proj.weight is on cuda:0
model.layers.1.self_attn.v_proj.weight is on cuda:0
model.layers.1.self_attn.o_proj.weight is on cuda:0
model.layers.1.mlp.gate_proj.weight is on cuda:0
model.layers.1.mlp.up_proj.weight is on cuda:0
model.layers.1.mlp.down_proj.weight is on cuda:0
model.layers.1.input_layernorm.weight is on cuda:0
model.layers.1.post_attention_layernorm.weight is on cuda

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Yes.


In [17]:
# Define video input path and the zero-shot prompt query
video_path = "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/File1.mp4"
query = '''
Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: "Yes" or "No".

Answer:
'''

# Run the model's chat function
out, history = model.chat(
    video_path=video_path,
    query=query,
    tokenizer=tokenizer,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print("Assistant:\n", out)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Yes.


In [18]:
# Define video input path and the zero-shot prompt query
video_path = "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/File2.mp4"
query = '''
Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: "Yes" or "No".

Answer:
'''

# Run the model's chat function
out, history = model.chat(
    video_path=video_path,
    query=query,
    tokenizer=tokenizer,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print("Assistant:\n", out)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 No.


In [19]:
# Define video input path and the zero-shot prompt query
video_path = "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/File3.mp4"
query = '''
Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: "Yes" or "No".

Answer:
'''

# Run the model's chat function
out, history = model.chat(
    video_path=video_path,
    query=query,
    tokenizer=tokenizer,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print("Assistant:\n", out)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Yes.


In [20]:
# Define video input path and the zero-shot prompt query
video_path = "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/File4.mp4"
query = '''
Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: "Yes" or "No".

Answer:
'''

# Run the model's chat function
out, history = model.chat(
    video_path=video_path,
    query=query,
    tokenizer=tokenizer,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print("Assistant:\n", out)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Yes.


: 